In [14]:
import os
import re
import pandas as pd
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
data = []
root = './blogs'
for fil in os.listdir(root):
    try:
        meta = fil.split('.')[1:3]
        content = open(os.path.join(root, fil), "r", encoding='utf-8', errors='ignore').read()
        data.append(meta+[content])
    except:
        print(os.path.join(root, fil))

In [3]:
data = pd.DataFrame(data, columns=['gender', 'age', 'posts'])
data.head()

,gender,age,posts
0,female,37,"<Blog>\n\n<date>31,May,2004</date>\n<post>\n\n..."
1,female,17,"<Blog>\n\n<date>23,November,2002</date>\n<post..."
2,male,23,"<Blog>\n\n<date>19,June,2004</date>\n<post>\n\..."
3,female,25,"<Blog>\n\n\n\n<date>31,May,2004</date>\n<post>..."
4,male,25,"<Blog>\n\n<date>05,July,2003</date>\n<post>\n\..."


In [51]:
data['posts'][0][:40]

'<Blog>\n\n<date>31,May,2004</date>\n<post>\n'

In [4]:
# нормализовать
# я не имею НИ МАЛЕЙШЕГО ПОНЯТИЯ что значит "обратить внимание на тэги" и просто удаляю их к чертям
data['normalized'] = data['posts'].map(lambda x: re.sub('</?[(?:Blog)|(?:post)]+>|<date>[^<]+</date>', '', x))
data['normalized'] = data['normalized'].map(lambda x: re.sub('\s{2,}', ' ', x).strip())

In [53]:
data['normalized'][0][:40]

'Well, everyone got up and going this mor'

In [7]:
# векторизовать
cv = CountVectorizer(preprocessor=WordNetLemmatizer().lemmatize, stop_words='english', min_df=2, max_df=0.95)
cv_matrix = cv.fit_transform(data['normalized'])
tfidf_transformer = TfidfTransformer()
tfidf = tfidf_transformer.fit_transform(cv_matrix)

In [9]:
X_train, X_test, y_train, y_test = train_test_split(tfidf, data['gender'])

например, построить частотные списки и сравнить их (достаточно ли матрицы терм-документ с ненормализованными вхождениями слов?)

In [ ]:
# НА*&#!@

In [ ]:
# лес
forest = RandomForestClassifier(random_state=6)
params = {}
gs = GridSearchCV(forest, params)
gs = gs.fit(X_train, y_train)
# здесь про параметры
prediction = gs.predict(X_test)
print(classification_report(y_test, prediction))
print(confusion_matrix(y_test, prediction))

In [ ]:
# байес
bayes = MultinomialNB()
params = {}
gs = GridSearchCV(bayes, params)
gs = gs.fit(X_train, y_train)
# здесь про параметры
prediction = gs.predict(X_test)
print(classification_report(y_test, prediction))
print(confusion_matrix(y_test, prediction))

In [ ]:
# логрег
lgrg = LogisticRegression()
params = {}
gs = GridSearchCV(lgrg, params)
gs = gs.fit(X_train, y_train)
# здесь про параметры
prediction = gs.predict(X_test)
print(classification_report(y_test, prediction))
print(confusion_matrix(y_test, prediction))